In [985]:
import os
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
from skimage import io
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, utils
import torchvision.transforms as T
import matplotlib.pyplot as plt

In [986]:
class imgDataset(Dataset):
    def __init__(self, dir, transform=None):
        self.data = []
        for root, dirs, files in os.walk(dir):
            for file in files:
                if not file.startswith('.'):
                    img = transform(io.imread(os.path.join(root, file)))
                    if img.size()[0] > 3:
                        img = torch.mean(img, dim=0, keepdim=True)
                    if img.size()[0] > 1:
                        img = T.Grayscale(1)(img)
                    cancer = int('yes' in root)
                    self.data.append({'image': img, 'cancer': cancer, 'name': file})
        random.shuffle(self.data)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self.data[idx]
    def __len__(self):
        return len(self.data)

In [987]:
dir = './archive'
batch_size = 4
transform = T.Compose([T.ToTensor(), T.Resize((200,200), antialias=True)])
dataset = imgDataset(dir, transform)
k = int(0.8 * len(dataset))
train = dataset[0:k]
test = dataset[k:]
train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test, batch_size=batch_size, shuffle=True)

In [988]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)
print(f'Using {device} device')

Using mps device


In [989]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Linear(200*200, 64),
            nn.ReLU(),
            nn.Linear(64, 512),
            nn.ReLU(),
            nn.Linear(512,1024),
            nn.ReLU(),
            nn.Linear(1024,2)
        )
    def forward(self, x):
        x = nn.Flatten(1)(x)
        x = self.stack(x)
        return x

In [990]:
model = Net()

In [991]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, x in enumerate(dataloader):
        pred = model(x['image'])
        loss = loss_fn(pred, x['cancer'])

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 5 == 0:
            loss, current = loss.item(), (batch + 1) * len(x['image'])
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [992]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for x in dataloader:
            pred = model(x['image'])
            test_loss += loss_fn(pred, x['cancer']).item()
            correct += (pred.argmax(1) == x['cancer']).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [993]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


TypeError: linear(): argument 'input' (position 1) must be Tensor, not Flatten

In [ ]:
n = random.randint(0,50)
x = test_dataloader.dataset[n]
pred = nn.Softmax(dim = 0)(model(x['image']))
plt.imshow(x['image'].permute(1,2,0))
print(pred, x['cancer'], x['name'])

RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x64 and 1024x120)